In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import imageio

In [2]:
file_path = 'data/tracking_week_4.csv'  # Replace with your actual file path
game_id = 2022092900  # Replace with the desired game ID
play_id = 57  # Replace with the desired play ID
output_file = 'output/football_play_animation.gif'

In [3]:
def load_and_filter_data(file_path, game_id, play_id):
    df = pd.read_csv(file_path)
    play_data = df[(df['gameId'] == game_id) & (df['playId'] == play_id)]
    return play_data

def create_football_field(ax):
    ax.set_facecolor('green')
    
    # Add yard lines
    for yard in range(10, 110, 10):
        ax.axvline(yard, color='white', linestyle='-', linewidth=2)
        if yard == 50:
            ax.text(yard, 5, str(yard), color='white', ha='center')
        elif yard < 50:
            ax.text(yard, 5, str(yard), color='white', ha='center')
            ax.text(120-yard, 5, str(yard), color='white', ha='center')
    
    # Add end zones
    ax.axvspan(0, 10, facecolor='blue', alpha=0.3)
    ax.axvspan(110, 120, facecolor='red', alpha=0.3)

def update_play(frame, play_data, scatter_dict, ball_scatter, frame_text, ax, full_field_view_mode):
    frame_data = play_data[play_data['frameId'] == frame]
    
    for team in scatter_dict.keys():
        if team == 'football':
            continue
        team_data = frame_data[frame_data['club'] == team]
        scatter_dict[team].set_offsets(team_data[['x', 'y']])
        # Update jersey numbers
        for i, player in team_data.iterrows():
            ax.annotate(str(int(player['jerseyNumber'])), 
                        (player['x'], player['y']),
                        xytext=(0, 5),
                        textcoords='offset points',
                        ha='center',
                        fontsize=8)
    
    ball_data = frame_data[frame_data['nflId'].isna()]
    if not ball_data.empty:
        ball_scatter.set_offsets(ball_data[['x', 'y']])

    frame_text.set_text(f"Frame: {frame}")

    if full_field_view_mode:
        # Setting field limits
        padding = 5
        ax.set_xlim(-padding, 120+padding)
        ax.set_ylim(-padding, 53.3+padding)

    else:
        # Adjust the field of view
        x_min, x_max = frame_data['x'].min(), frame_data['x'].max()
        y_min, y_max = frame_data['y'].min(), frame_data['y'].max()
        
        # Add some padding
        padding = 10
        ax.set_xlim(max(0, x_min - padding), min(120, x_max + padding))
        ax.set_ylim(max(0, y_min - padding), min(53.3, y_max + padding))

    
    # Clear previous annotations
    for artist in ax.artists + ax.texts:
        if artist != frame_text:
            artist.remove()
    
    return list(scatter_dict.values()) + [ball_scatter, frame_text]

def create_play_animation(play_data, output_file, full_field_view_mode=False):
    fig, ax = plt.subplots(figsize=(12, 6))
    create_football_field(ax)
    
    # Determine all unique teams
    teams = play_data['club'].unique()
    
    # Create initial scatter plots for each team and the ball
    scatter_dict = {team: ax.scatter([], [], s=100, label=team) for team in teams}
    ball_scatter = ax.scatter([], [], color='brown', s=50, label='Ball')
    
    # Add frame text
    frame_text = ax.text(0.02, 0.95, '', fontsize=10, transform=ax.transAxes)
    
    # Create the animation
    anim = animation.FuncAnimation(
        fig, update_play, frames=play_data['frameId'].unique(),
        fargs=(play_data, scatter_dict, ball_scatter, frame_text, ax, full_field_view_mode),
        interval=100, blit=False
    )
    
    # Save the animation as a GIF
    anim.save(output_file, writer='pillow', fps=10)
    plt.close(fig)

In [4]:
play_data = load_and_filter_data(file_path, game_id, play_id)
create_play_animation(play_data, output_file, full_field_view_mode=False)

print(f"Animation saved as {output_file}")

Animation saved as football_play_animation.gif
